In [1]:
import os
import sys
import openai
import pandas as pd
import numpy as np
import argparse
import torch
import operator
from sklearn.metrics import classification_report
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
data_path="../../data/"
outputs_path="../../outputs/"
train_set=pd.read_csv(outputs_path+"general/train_set_final.csv")
df_test_final=pd.read_csv(outputs_path+"general/test_set_final.csv")
df_test_final=df_test_final.iloc[:2000].reset_index(drop=True)

In [4]:
df_test_final=df_test_final.reset_index(drop=True)

df=df_test_final.reset_index(drop=True).copy()

In [7]:
train_set.shape

(2500, 16)

## few shot

In [8]:
## french sample
train_set=train_set.loc[train_set.ID!='grabski-etal-2012-controle'].reset_index(drop=True)

In [9]:
shots=2

In [10]:
## balance
train_set_pos=train_set.loc[train_set.label==1].reset_index(drop=True)
train_set_neg=train_set.loc[train_set.label==0].reset_index(drop=True)

#train_set_pos_sample=train_set_pos.sample(n=int(shots/2),random_state=42).copy()
#train_set_neg_sample=train_set_neg.sample(n=int(shots/2),random_state=42).copy()
train_set_pos=train_set_pos.loc[:int(shots/2)-1,]
train_set_neg=train_set_neg.loc[:int(shots/2)-1,]


few_shots=pd.concat([train_set_pos,train_set_neg]).reset_index(drop=True)
#few_shots=pd.concat([train_set_pos,train_set_neg]).sample(frac=1,random_state=42).reset_index(drop=True)
#few_shots=pd.concat([train_set_pos,train_set_neg]).sort_index(kind='merge')

In [11]:
## build examples
examples=""
for i,d in few_shots.iterrows():
    outcome=""
    if d['label']==1:
        outcome="Yes"
    else:
        outcome="No"
    examples+="Paper: "+d['title_clean']+" \nAnswer: "+outcome+"\n"
print(examples)

Paper: Aspect-based Document Similarity for Research Papers 
Answer: Yes
Paper: Neural Stylistic Response Generation with Disentangled Latent Variables 
Answer: No



In [12]:
preprompt="Are these papers contributing to the UN Sustainable Development Goals or helping society?\n"
#df=df.assign(statement=preprompt+examples+df.text+" =>")
df=df.assign(statement=preprompt+examples+"Paper: "+df.title_clean+" \nAnswer: ")

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['statement']
    print(input_prompt)
    print("original label: ",d['label'],d['goal1_raw'],d['goal2_raw'],d['goal3_raw'])
    print()

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['statement']
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=50,logprobs=10)
    dict_norm={}
    dict_uniques={}
    ## new part, get the first element where there the most probable is not a break of line
    for choices in completion['choices'][0]["logprobs"]['top_logprobs']:
        choice=dict(choices)
        max_val=max(choice.items(), key=operator.itemgetter(1))[0]
        if "\n" not in max_val:
            elements=choices
            break
    for e in elements:
        e_modified=e.lower().lstrip(' ')
        if e_modified in dict_uniques:
            dict_uniques[e_modified]=dict_uniques[e_modified]+np.exp(elements[e])
        else:
            dict_uniques[e_modified]=np.exp(elements[e])

    if ('no' in dict_uniques.keys()) and ('yes' in dict_uniques.keys()):
        dict_norm={'no':dict_uniques['no'],'yes':dict_uniques['yes']}
    elif ('no' in dict_uniques.keys()):
        dict_norm={'no':dict_uniques['no'],'yes':0}
    elif ('yes' in dict_uniques.keys()):
        dict_norm={'no':0,'yes':dict_uniques['yes']}

    factor=1.0/sum(dict_norm.values())
    for k in dict_norm:
        dict_norm[k] = dict_norm[k]*factor    

    df.loc[i,'full_prompt']=input_prompt
    df.loc[i,'GPT3_response']=completion.choices[0].text
    df.loc[i,'proba_1']=dict_norm['yes']
    df.loc[i,'proba_0']=dict_norm['no']
    print(input_prompt)
    #print(d['text'])
    print(completion.choices[0].text)
    print("proba yes: ",dict_norm['yes'])
    print("original label: ",d['label'],d['goal1_raw'],d['goal2_raw'],d['goal3_raw'])
    print("#########################################")
    print()

In [16]:
df=df.assign(prediction_proba=np.where(df.proba_1>0.5,1,0))

In [23]:
df=df.assign(textual_pred=np.where(((df.GPT3_response.str.lower().str.contains("yes")) | 
                                     (df.GPT3_response.str.lower().str.contains("is contrib"))
                                     ),1,0))

In [17]:
df.prediction_proba.value_counts()

0    1383
1     617
Name: prediction_proba, dtype: int64

## negative then positive

In [18]:
## 2 shot paper, answer
print(classification_report(df.label,df.prediction_proba,digits=4))

              precision    recall  f1-score   support

           0     0.9262    0.7379    0.8214      1736
           1     0.2626    0.6136    0.3678       264

    accuracy                         0.7215      2000
   macro avg     0.5944    0.6758    0.5946      2000
weighted avg     0.8386    0.7215    0.7615      2000



In [18]:
## 4 shot paper, answer
print(classification_report(df.label,df.prediction_proba,digits=4))

              precision    recall  f1-score   support

           0     0.9267    0.8082    0.8634      1736
           1     0.3148    0.5795    0.4080       264

    accuracy                         0.7780      2000
   macro avg     0.6207    0.6939    0.6357      2000
weighted avg     0.8459    0.7780    0.8033      2000



In [18]:
## 8 shot paper, answer
print(classification_report(df.label,df.prediction_proba,digits=4))

              precision    recall  f1-score   support

           0     0.8913    0.9591    0.9240      1736
           1     0.4621    0.2311    0.3081       264

    accuracy                         0.8630      2000
   macro avg     0.6767    0.5951    0.6160      2000
weighted avg     0.8347    0.8630    0.8427      2000



In [18]:
## 16 shot paper, answer
print(classification_report(df.label,df.prediction_proba,digits=4))

              precision    recall  f1-score   support

           0     0.9048    0.9470    0.9254      1736
           1     0.4973    0.3447    0.4072       264

    accuracy                         0.8675      2000
   macro avg     0.7010    0.6459    0.6663      2000
weighted avg     0.8510    0.8675    0.8570      2000



In [20]:
## 32 shot paper, answer
print(classification_report(df.label,df.prediction_proba,digits=4))

              precision    recall  f1-score   support

           0     0.9085    0.9038    0.9062      1736
           1     0.3883    0.4015    0.3948       264

    accuracy                         0.8375      2000
   macro avg     0.6484    0.6527    0.6505      2000
weighted avg     0.8398    0.8375    0.8387      2000



In [27]:
df.to_csv(outputs_path+"sg_classifier/gpt3_32_shot_sg_prompt.csv",index=False)